# Data collection

In [52]:
import serial
import csv

ser = serial.Serial('COM9', 9600)  # Replace with the correct port name

with open('rawdata.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z'])
    
    for i in range(150):
        #line = ser.readline().decode().strip()
        try:
            line = ser.readline().decode().strip()
            values = line.split(',')
        except:
            # ignore the error and continue
            continue
        print(line)
        values = line.split(',')
        if len(values) != 6:
            continue
        try:
            accel_x = float(values[0])
            accel_y = float(values[1])
            accel_z = float(values[2])
            gyro_x = float(values[3])
            gyro_y = float(values[4])
            gyro_z = float(values[5])
        except ValueError:
            continue
        
        csvwriter.writerow([accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z])
        
ser.close()

3.96,0.17,-6.37,0.00,0.26,-0.10
0.40,6.85,-16.32,-0.33,-1.34,0.69
3.00,-0.32,-10.04,-4.08,-0.24,-0.78
-0.04,3.92,-7.99,1.31,-0.41,-1.39
0.53,7.61,-7.97,0.37,2.60,1.48
1.60,0.16,-8.95,-0.16,1.14,0.03
-3.98,9.38,-10.91,1.36,1.03,-0.30
1.82,3.37,-10.41,-0.44,-0.00,0.86
2.44,-3.37,-8.51,0.98,0.75,-0.89
-4.76,7.08,-13.05,-0.28,-1.01,0.92
2.82,-0.32,-10.52,-2.52,-2.16,1.37
6.45,4.00,-11.52,1.35,0.65,-2.52
1.22,5.31,-12.43,0.05,-0.78,0.84
8.11,-0.24,-9.18,1.03,-0.83,-1.17
0.50,3.22,-13.01,0.23,-0.59,-0.99
1.95,5.99,-11.96,-0.83,0.50,-0.10
2.17,-1.72,-6.57,-0.82,0.91,0.48
-0.91,15.60,-14.51,3.73,4.19,0.14
0.84,3.98,-9.75,-1.49,1.55,1.75
5.28,0.08,-12.66,1.67,2.06,-1.43
3.56,4.46,-12.37,-0.22,0.62,0.58
7.63,0.86,-10.41,0.63,0.02,-2.00
-1.65,6.17,-14.36,-0.71,-1.99,1.11
5.63,-1.68,-9.35,-1.32,-1.27,0.37
-2.42,10.42,-11.18,1.75,2.04,-0.87
-1.83,6.55,-9.23,-1.44,0.94,1.27
6.21,-1.72,-8.39,0.61,0.69,-1.67
-5.15,6.26,-13.98,-0.14,-1.21,0.33
7.10,1.95,-9.69,-1.80,1.28,2.62
2.12,1.29,-10.14,1.28,1.05,

In [5]:
ser.close()

# Preprocessing

In [63]:
import pandas as pd
import numpy as np

#insert raw data
df = pd.read_csv('rawdata.csv')

# calculate magnitude of acceleration readings
df['Acc_Magnitude'] = df.apply(lambda x: np.sqrt(x[0]**2 + x[1]**2 + x[2]**2), axis=1)

# calculate magnitude of gyro readings
df['Gyro_Magnitude'] = df.apply(lambda x: np.sqrt(x[3]**2 + x[4]**2 + x[5]**2), axis=1)

# extracting the last two columns
dfm=df.iloc[:, -2:]

# splitting the dataframe to multiple dataframes
dfs = []
fsize = 15
for i in range(0, len(df), fsize):
    dfs.append(dfm.iloc[i:i+fsize])

# print the extracted dataframes
for i, df_5rows in enumerate(dfs):
    globals()[f"df{i+1}"] = df_5rows
    
# Calculate the average of each DataFrame
avg_list = []
for df in dfs:
    avg = df.mean()
    avg_list.append(avg)

# Create a new DataFrame to store the averages
avg_df = pd.DataFrame(avg_list)

# GET AVG 

with open('avg_df3.csv', 'a') as f:
    avg_df.to_csv(f, header=True, index=False)

In [68]:
type(avg)
avg

Acc_Magnitude     13.101425
Gyro_Magnitude     2.741966
dtype: float64

In [65]:
avg_df

,Acc_Magnitude,Gyro_Magnitude
0,11.981759,1.888485
1,13.057142,2.336517
2,11.781431,1.553970
3,11.954950,1.552885
4,11.640909,1.904781
5,12.731665,2.003159
6,12.492987,1.969181
7,13.045823,1.612434
8,11.932348,2.355723
9,13.101425,2.741966


# *KNN*

## * Labeling data for training

In [48]:
import pandas as pd
import numpy as np

dst = pd.read_csv('avg_st1.csv')
ddy = pd.read_csv('avg_dy2.csv')

dst['label']=np.zeros(10)
ddy['label']=np.ones(10)

dtr = pd.concat([dst, ddy], ignore_index=True)
dtr.to_csv('train.csv', index=False)

## * Training KNN

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv('train.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=42)

# Create a KNN classifier with k=5

knn = KNeighborsClassifier(n_neighbors=7)

# Train the classifier on the training data
knn.fit(X_train, y_train)

# Use the classifier to predict the labels of the test data
y_pred = knn.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

Accuracy: 1.0


In [50]:
print(y_test)
print(y_pred)

0     0.0
17    1.0
15    1.0
1     0.0
Name: label, dtype: float64
[0. 1. 1. 0.]


## * Exporting the model

In [51]:
from joblib import dump, load

dump(knn, 'model.pkl')

['model.pkl']

# Importing the model and predicting

In [59]:
import joblib

knn_model = joblib.load('model.pkl')

# new data for prediction
new_data = pd.read_csv('avg_df3.csv')

# Make predictions on the new data using the loaded KNN model
predictions = knn_model.predict(new_data)

In [60]:
predictions

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 1.])

In [10]:
type(predictions)

numpy.ndarray

In [88]:
'''

END


'''

'\n\nEND\n\n\n'

In [80]:
av1=pd.DataFrame([avg])

In [81]:
av1

,Acc_Magnitude,Gyro_Magnitude
0,13.101425,2.741966


In [83]:
import joblib

knn_model = joblib.load('model.pkl')

# new data for prediction
new_data = av1

# Make predictions on the new data using the loaded KNN model
predictions = knn_model.predict(new_data)
predictions

array([1.])

In [84]:
type(predictions)

numpy.ndarray

In [86]:
predictions[0]

1.0